# Wikipedia bitaxonomy

In this notebook, three path-based measures (path similarity, Leacock & Chodorow, Wu & Palmer) for Wikipedia bitaxonomy. 
<br> 


## Imports

In [ ]:
from xml.dom import minidom
import math
import pandas as pd
from scipy.stats import pearsonr, spearmanr, kendalltau

In [ ]:
rpages = {}
for p in pages.keys():
    p1 = pages.get(p)
    for page in p1: 
        if 'HYPERNYM' in page.keys():
            hyps = page.get('HYPERNYM')
            for h in hyps:
                if h in rpages.keys():
                    val = rpages.get(h)
                    val.append(p)
                    rpages[h] = val
                else:
                    rpages[h]=[p]

## Defining the methods

In [ ]:
def calcEdges(s1, s2): 
    """
    returns edge count of the path 
    
    """
    if s1[-1] == s2[-1]:
        if set(s1) == set(s2): # two nodes have the same hypernym, so 2 edges in total / \
            return 2
        
        if set(s1).issubset(set(s2)): # if one is part of the other
            return len(s2) - len(s1) + 2
        
        if set(s2).issubset(set(s1)): # same logic as the previous
            return len(s1)-len(s2) +2
        
        c = set(s1).intersection(set(s2)) # finding intersection of those lists
        l1 = len(s1) - len(c) + 1 
        l2 = len(s2) - len(c) + 1
        return l1+l2
    
    return 1000 # if no intersection between those paths were found, return 1000
     
    
def calcNodes(s1, s2): 
    """
    returns nodes count on the path
    
    """
    return calcEdges(s1, s2) + 1


def PS(s1, s2): 
    """ 
    path similarity, calculated as ps=1/(minPath(s1,s2)+1)
    
    """ 
    nodes = calcNodes(s1, s2)

    ps = 1/nodes
    return ps


def LC(s1, s2, depth): 
    """
    Leacock & Chodorow, calculated as -log(minPath(s1, s2)/(2*(max_graph_depth)))
    
    """
    l = calcNodes(s1, s2)

    lc = -math.log(l/(2*depth))
    return lc


def intersection(lst1, lst2): 
    """
    returns the first same value from 2 lists
    
    """
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 


def depth(lcs, s):
    """
    returns the depth (from root to some node)
    
    """
    i = s.index(lcs)
    slice_s = s[i:]
    return len(slice_s)


def length(s1, s2): 
    lst = len(intersection(s1, s2))
 
    l1 = s1[:-lst]
    l2 = s2[:-lst]
    return len(l1)+1, len(l2)+2
    
    
def lcs(s1, s2):
    """
    returns the least common subsumer 
    
    """
    lst = intersection(s1, s2)
    return lst[0]
    
    
def WUP(s1, s2):
    """
    Wu & Palmer, calculated as ( 2*depth(lcs(s1,s2)) ) / ( depth(s1) + depth(s2) )
    
    """
    
    return (2*depth(lcs(s1, s2), s1)/(length(s1, s2)[0] + length(s1, s2)[1]+ 2*depth(lcs(s1, s2), s1)) )


def findDepth(categories, pages):
    """
    returns the dictionary, where the maximum path length from root to leaf in category taxonomy is saved
    
    """
    longs = {}
    
    for l in pages.keys():
        all_path = all_paths(categories, pages, l)
        
        for p in all_path:
            length = len(p)
            root = p[-1]
            
            if root not in longs.keys():
                longs[root] = length
            else: 
                if longs.get(root) < length: 
                    longs[root] = length
    return longs


def all_paths(categories, pages, w1):
    """
    returns all possible paths to a root node (first node is the page node, using cross_links it is moved to category taxonomy)
    
    """
    
    visited = []
    paths = []
    
    if w1 in pages.keys():        
        cs = [ link for page in  pages.get(w1) if 'CROSS_LINK' in page.keys() for link in page.get('CROSS_LINK')]
    else: 
        return []
    queue = [[w] for w in cs]
    
    while queue: 
        path = queue.pop(0)
        val = path[-1]
        neighbours = getAllEdges(val, categories)
        
        if neighbours == set():
            paths.append(path)
            
        for n in neighbours:
            new = list(path)
            new.append(n)
            queue.append(new)
    return [p for p in paths if p != [w1]]


def all_paths_pages(pages, w1):
    """
    returns all possible paths to a root node, only using page taxonomy
    
    """
    visited = []
    paths = []
    queue = [[w1]]
    
    while queue: 
        path = queue.pop(0)
        val = path[-1]
        neighbours = getAllEdges(val, pages)
        
        if neighbours == set():
            paths.append(path)
            
        for n in neighbours:
            new = list(path)
            new.append(n)
            queue.append(new)
            
    return [p for p in paths if p != [w1]]


def minPath(h1, h2):
    """
    returns the paths to root, with what minimum path between two nodes is found
    
    """
    shortest = 1000
    p1 = []
    p2 = []
    for h1_path in h1: 
        for h2_path in h2: 
            path_len= calcEdges(h1_path, h2_path)
            if path_len < shortest: 
                shortest = path_len
                p1 = h1_path
                p2 = h2_path
 
    if shortest == 1000: 
        return -1, -1
    return p1, p2


def findDepthPages(pages):
    """
    returns the maximum path lengths from root to leaf in page taxonomy
    
    """
    
    longs = {}
    for l in pages.keys():
        all_path = all_paths_pages(pages, l)
        for p in all_path:
            length = len(p)
            root = p[-1]
            if root not in longs.keys():
                longs[root] = length
            else: 
                if longs.get(root) < length: 
                    longs[root] = length
    return longs


def getAllEdges(w, dic):
    """
    returns the hypernyms of a node
    
    """
    
    edges = []
    if w in dic.keys():
        val = dic.get(w)
        for v in val:
            if 'HYPERNYM' in v.keys():
                hyp = v.get('HYPERNYM')
                edges.extend(hyp)
    return set(edges)

def calc_results(df):
    """
    returns dataframe containing correlations scores
    
    """
    results = pd.DataFrame(columns=["measure", "sim_set", "pearson", "spearman", "kendall"]) 
    results = evaluate(df, results, "ESL", "PS")
    results = evaluate(df, results, "ESL", "LC")
    results = evaluate(df, results, "ESL", "WUP")
    results = evaluate(df, results, "SL", "PS")
    results = evaluate(df, results, "SL", "LC")
    results = evaluate(df, results, "SL", "WUP")
    return results
    
    
    
def evaluate(df, results, sim_set, measure):
    """
     calculates correlation coefficients
    
    """
    pearson = round(pearsonr(df[sim_set], df[measure])[0], 3)
    spearman = round(spearmanr(df[sim_set], df[measure])[0], 3)
    kendall = round(kendalltau(df[sim_set], df[measure])[0],3)
    results = results.append({"measure":measure, "sim_set":sim_set, "pearson":pearson, "spearman":spearman, "kendall":kendall},
                            ignore_index=True)
    return results


## XML file parsing

Bitaxonomy is in a XML file. This file contains doc elements, each doc element is a Wikipedia page or category. Hypernyms are defined for each group. Also for every page, there a so called cross_links to wikipedia categories. 

In [ ]:
# parsing the XML file 
mydoc = minidom.parse('ET_taxonomy.xml')
items = mydoc.getElementsByTagName('doc')

In [ ]:
# seperating pages and categories 
pages = {}
categories = {}
alls = {}
for item in items: 
    name = ""
    fields = item.getElementsByTagName('field')
    fieldDict = {}
    for field in fields: 
        fname = field.attributes['name'].value # PAGE/CATEGORY/HYPERNYM/HYPERLINK
        vals = field.getElementsByTagName('val') #<val>
        values = []
        for val in vals: 
            value = val.firstChild.data.strip().lower()
            
            if fname == "PAGE" or fname == "CATEGORY":
                fieldDict["FNAME"] = fname 
                name = value
            else:
                values.append(value)
                
        if values != []:
            fieldDict[fname] = values
            
    if len(fieldDict.keys()) != 1: 
        if name in alls.keys(): 
            d = alls.get(name)
            d.append(fieldDict)
            alls[name] = d
        else:
            alls[name] = [fieldDict]
            
    if name in categories.keys() and fieldDict["FNAME"]=="CATEGORY":
        d = categories.get(name)
        d.append(fieldDict)
        categories[name] = d
    elif name in pages.keys() and fieldDict["FNAME"]=="PAGE":
        d = pages.get(name)
        d.append(fieldDict)
        pages[name]=d
    
    else:
        if fieldDict["FNAME"] == "CATEGORY":
            categories[name] = [fieldDict]
        elif fieldDict["FNAME"] == "PAGE":
            pages[name]=[fieldDict]


## Calculating similarity and correlations
First, similarity is calculated beween the EstSimLex-999 pairs and then the correlations between those and humans scores are calculated. 
<br>
First, the needed data files are loaded and some variables are initiated. 

In [ ]:
# loading the EstSimLex-999 data set
file_name = "Ratings.xlsx"
data = pd.read_excel(file_name)
# finding max depths of  page taxonomy
depths_graph_pages = findDepthPages(pages)

Page taxonomy is used to calculate similarity. 

In [ ]:
# dataframe, where the similarity scores will be saved
similarity_scores_pages = pd.DataFrame(columns=["sõna1", "sõna2","PS", "LC", "WUP", "ESL", "SL"])

for i, row in data.iterrows():
    w1 = row["sõna 1"]
    w2 = row["sõna 2"]
    hyp1 = all_paths_pages(pages, w1)
    hyp2 = all_paths_pages(pages, w2)
    esl = row["Average"]
    sl = row["SimLex999"]
    if hyp1 != [] and hyp2 != []:
        p1,p2= minPath(hyp1, hyp2)
        
        if p1 != -1:
            path_sim = PS(p1, p2)
            d = depths_graph_pages.get(p1[-1])+1
            l = LC(p1, p2, d)
            w = WUP(p1, p2)
            similarity_scores_pages = similarity_scores_pages.append({"sõna1":w1, "sõna2":w2, "PS":path_sim, "LC":l, "WUP":w, "ESL":esl,
                                                          "SL":sl}, ignore_index=True)
            


results_page = calc_results(similarity_scores_pages)


In [ ]:
# saving results to excel
results_page.to_excel("pages_tax_results.xlsx")